# HTML、タグ情報抽出

In [24]:
# -*- coding: utf-8
import urllib.request
import html.parser as hp

TRUNK_URL = "https://matome.naver.jp"
TARGET_URL = "https://matome.naver.jp/odai/2153448009006325301"
ANSWER_URL = "https://matome.naver.jp/odai/2153448009006325301?page=2"

class Parser(hp.HTMLParser):
    """
    HTMLParserを継承したパーサー
    """
    # result_tagsがパーサーの取得結果となる
    result_tags = {}
    # iがidでKEY情報となる。1回タグ情報を取るたびに加算される。
    i = 0
    def handle_starttag(self, tag, attrs):
        """
        各タグの最初の文字列を処理する
        ex) <html></html> => html
        """
        self.i += 1
#         self.result_tags[self.i] = [tag,attr]
        self.result_tags[self.i] = [tag]

    def handle_data(self, data):
        """
        各タグの中身データ部を処理する
        KEYであるID
        """
        try:
            #まだTagしか登録されていない場合のみData(文字列）を登録
            if len(self.result_tags[self.i]) == 1:
                self.result_tags[self.i].append(data.strip())
        except:
            pass

class LinkParser(Parser):
    """
    そのページにリンクが貼られているパス情報を取得するためのパーサー
    """
    result_links = {}
    i = 0
    def handle_starttag(self, tag, attrs):
        self.i += 1
        if tag == "a" :
            self.result_links[self.i] = [attrs]

    def handle_data(self, data):
        pass

def get_links(url):
    """
    URLから、そのページにリンクが貼られているパス情報（そのまま、相対パス含）を取得
    """
    html = get_html(url)
    link_parser = LinkParser()
    link_parser.feed(html)
    link_parser.close()
    return link_parser.result_links

# TARGET_URL = "https://matome.naver.jp/odai/2153448009006325301"
def get_absolute_links(target_url):
    """
    URLから、そのページにリンクが貼られている絶対パス情報を取得
    """
    absolute_links = []
    links = get_links(target_url)
    for key, link in links.items():
        link_dict = dict(link[0])
        if "href" in link_dict:
            link_path =  link_dict["href"]
            if link_path.startswith("http"):
                absolute_links.append(link_path)
            elif link_path.startswith("/"):
                absolute_links.append(TRUNK_URL + link_path)
            else:
                absolute_links.append(target_url + link_path)
    return absolute_links
    
def get_html(url):
    """
    URLからHTML情報を取得
    """
    # HTMLファイルを開く
    data = urllib.request.urlopen(url)
    # HTMLの取得      
    html = data.read()
    data.close()
    return html.decode('utf-8')
        
def get_result_tags(url):
    """
    URLからタグ情報を取得
    """
    # 取得先URLが引数
    html = get_html(url)
    # HTML解析
    parser = Parser()
    parser.feed(html)
    # 終了処理
    parser.close()
    return parser.result_tags

# if __name__ == "__main__":
#     tags = get_result_tags("https://matome.naver.jp/odai/2153448009006325301")
#     absolute_links = adjust_absolute_links("https://matome.naver.jp/odai/2153448009006325301")

# タグからリンク候補先の一致率比較

In [25]:
def extract_useful_tags(tags):
    """
    タグ群から一致率判定に利用できるタグ情報を取得
    """
    result = {}
    for k, v in tags.items():
        if len(v) == 2 and v[1] != "":
            result[k] = v
    return result

def get_useful_tags(url):
    """
    URLから一致率判定に利用できるタグ情報を取得
    """
    tags = get_result_tags(url)
    tags = extract_useful_tags(tags)
    return tags

In [26]:
def calculate_precision(source_tags, dst_tags):
    """
    一致率判定に利用できるタグ情報を取得
    計算式 
    
    比較元と同じ内容があるタグ数 / 比較先の情報があるタグ数
    """
    # 0から始まって同じ内容があれば加算される
    numer = 0
    #比較先の意味のあるタグ数が分母
    denom = len(dst_tags)
    for dst_tag in dst_tags.values():
        for source_tag in source_tags.values():
            if source_tag == dst_tag:
                numer = numer + 1
                break
    precision = numer / denom
    return precision

In [31]:
def calculate_pagination_probability(target_url=TARGET_URL):
    """
    対象のURLから、ページネーションの遷移先リンクの確率判定する
    """
    source_tags = get_useful_tags(target_url)
    precision_dict = {}
    absolute_links = [ANSWER_URL]
    absolute_links.extend(get_absolute_links(target_url))
    for absolute_link in absolute_links:
        try:
            print(absolute_link)
            dst_tags = get_useful_tags(absolute_link)
            precision = calculate_precision(source_tags, dst_tags)
            precision_dict[absolute_link] = precision
        except:
            continue
    return precision_dict

In [32]:
ans = calculate_pagination_probability()
sorted(ans.items(), key=lambda x: x[1],reverse=1)

https://matome.naver.jp/odai/2153448009006325301?page=2
https://matome.naver.jp/
https://matome.naver.jp/odai/new
https://account.matome.naver.jp/service/join?fromUrl=https%3A%2F%2Fmatome.naver.jp%2Fodai%2F2153448009006325301
https://matome.naver.jp/odai/2153448009006325301javascript:goLogin()
https://matome.naver.jp/
https://matome.naver.jp/news
https://matome.naver.jp/topic/1HioN
https://matome.naver.jp/topic/1LzKf
https://matome.naver.jp/topic/1Hill
https://matome.naver.jp/topic/1M7Zv
https://matome.naver.jp/topic/1LvBA
https://matome.naver.jp/shuffle/2153448009006325301
http://amanaimages.com/info/infoRF.aspx?SearchKey=10285001003
https://matome.naver.jp/odai/2153448009006325301
http://b.hatena.ne.jp/entry/https://matome.naver.jp/odai/2153448009006325301
https://matome.naver.jp/report/abuse?fromUrl=https://matome.naver.jp/odai/2153448009006325301
https://matome.naver.jp/profile/pinkswan999
https://matome.naver.jp/profile/pinkswan999
http://www.beans-fukushima.or.jp/2017/08/beans_in

{'http://b.hatena.ne.jp/entry/https://matome.naver.jp/odai/2153448009006325301': 0.2,
 'http://help.naver.jp/faq?serviceNo=4': 0.2952586206896552,
 'http://help.naver.jp/privacy/': 0.3300110741971207,
 'http://help.naver.jp/rules/': 0.3274531422271224,
 'http://navermatome-official.blog.jp/': 0.24653148345784417,
 'http://topics.smt.docomo.ne.jp/article/jijico/life/jijico-28336': 0.06296691568836713,
 'http://welcome-to-gettyimages.jp/licensed-by-gettyimages/': 0.12725225225225226,
 'http://www.beans-fukushima.or.jp/2017/08/beans_info20170823/': 0.29570552147239265,
 'http://www.jprime.jp/articles/-/6265': 0.04528763769889841,
 'http://www.mext.go.jp/ijime/detail/dial.htm': 0.3786295005807201,
 'https://account.matome.naver.jp/service/join?fromUrl=https%3A%2F%2Fmatome.naver.jp%2Fodai%2F2153448009006325301': 0.6042154566744731,
 'https://ameblo.jp/tuffy777/entry-12398419500.html': 0.1945080091533181,
 'https://asqmii.com/jijico/2017/09/08/articles28336.html': 0.34104046242774566,
 'http